In [3]:
import pandas as pd 
import numpy as np 
from pymongo import MongoClient
from sklearn.feature_extraction.text import CountVectorizer
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import os
import cPickle as pk
import pdb
import math

In [31]:
class properties:
    def __init__(self, db, name):
        self.name=name
        self.dat = pd.DataFrame(list(db.find()))
        if os.path.isfile(self.name+"_params.pk"):
            self.params = pk.load(open(self.name+"_params.pk","rb"))        
        self.stop_words= pk.load(open("stop_words.pk","rb"))
        
    def benchmark(self):
        if os.path.isfile(self.name+"_params.pk"):
            return self.params
        else:
            self.params = self.analyze()
            self.params['latitude'] = self.dat[self.get_relevcols('latitude')]
            self.params['longitude'] = self.dat[self.get_relevcols('longitude')]
            self.params['city'] = self.dat['city']
            #print self.params
            print "Writing to: ", self.name + "_parameters.scsv" 
            self.params.to_csv(self.name+"_parameters.scsv",encoding='utf-8',index=False,sep=';')
            print "Writing to: ", self.name + "_parameters.pk"
            pk.dump(self.params,open(self.name+"_params.pk","wb"))
        
        return self.params
    
    def get_relevcols(self, colname):
        return [col for col in self.dat.columns.values.str.lower() if colname in col.lower()]
    
    def get_df(self):
        #Extract column names
        sve_col = [col for col in self.dat.columns.values if 'name' in col.lower() or 'address' in col.lower() or 'latitude' in col.lower() or 'longitude' in col.lower() or "city" in col.lower()]
        sve_col.append('location')
        
        #Extract columns
        ndf = pd.DataFrame([self.dat[ccol].str.lower() if 'name' in ccol.lower() or 'address' in ccol.lower() or 'city' in ccol.lower() else self.dat[ccol] for ccol in sve_col]).T
        ndf.columns = map(unicode.lower, ndf.columns)
        return ndf
    
    def analyze(self):
        #Get name and address data
        namedat = self.dat[self.get_relevcols('name')]
        addr_dat = self.dat[self.get_relevcols('address')]

        #Create dataframe
        prop_data = pd.DataFrame({"name_bigrams": gramsplitter(namedat,2),
                                  "address_trigrams": gramsplitter(addr_dat,3)})

        return prop_data
    
    def gramclean(self,x):
        if x.isnull().all():
            return np.nan
        return ' '.join([word for word in x.str.replace('[^\w\s]','').str.split().tolist()[0] if word not in self.stop_words])
    
    #Clean data frame and generate ngrams
    def gramsplitter(self,df,n):
        df = df.apply(gramclean,axis=1)
        return df.map(lambda x: x if (isinstance(x,float)) else self.ngrammer(x.split(" "),n))
    
    def ngrammer(self,inplis, n):
        if len(inplis)<1:
            return np.nan
        return list(zip(*[inplis[i:] for i in range(n)]))
    
    

In [30]:
obj_ = properties(db.ean_properties,"ean")
obj_.benchmark()

TypeError: get_relevcols() takes exactly 2 arguments (3 given)

In [34]:
def get_relevcols(df, colname):
	return [col for col in df.columns.values if colname in col.lower()]

In [4]:
def dump4bigram(*args):
	for idx,subdf in enumerate(args):
		subdf[get_relevcols(subdf, 'name')].to_csv(repr(idx)+"_names.txt",sep='\n',encoding='utf-8',index=False,header=False)
		subdf[get_relevcols(subdf, 'address')].to_csv(repr(idx)+"_address.txt",sep='\n',encoding='utf-8',index=False,header=False)

In [5]:
def tokener(textS):
	#Remove punctuation
	def splitter(txt):
		tx1 = txt.str.replace('[^\w\s]','')
		return tx1.str.split()
	
	tdat = textS.apply(splitter,axis=1)
	tdatS = pd.Series(tdat.squeeze(),index=tdat.index)
	#Flatten Series
	flattdat = [item for sublist in tdatS.tolist() for item in sublist]
	#Get english+french+german stopwords (such as and, or, is, the, a etc.)
	eng_stop_words = set(stopwords.words("english")+['test','1234','various','addresses'])
	fr_stop_words = set(stopwords.words("french"))
	ger_stop_words = set(stopwords.words("german"))
	stop_wrds = eng_stop_words.union(fr_stop_words).union(ger_stop_words)
	
	#Remove all such stopwords
	flattdat = [w for w in flattdat if w.lower() not in stop_wrds]

	return flattdat

In [6]:
def exploretext(textS,nme):
	#Tokenize data
	flattdat = tokener(textS)
	nltext = nltk.Text(flattdat)
	
	#Print most common bigrams
	print nltext.collocations()
	
	#Plot cumulative frequency distribution of words
	fdistr = nltk.FreqDist(nltext)
	fdistr.plot(50,cumulative=True)
	plt.savefig("cuml_freq_indvwords_"+nme+".png")
	plt.close()

	#Plot freq distribution of number of letters in a token present
	lettfdistr = nltk.FreqDist([len(w) for w in nltext])
	lettfdistr.plot(50)
	plt.savefig("letr_freq_"+nme+".png")
	plt.close()

	#Plot bigrams 
	nlbigrams = list(nltk.bigrams(flattdat))
	print nlbigrams[:50]
	fd = nltk.FreqDist(nlbigrams)
	fd.plot(10, cumulative=True)
	plt.savefig("bigram_freq_"+nme+".png")
	plt.close()

In [7]:
#Produce n gram
def ngrammer(inplis, n):
	if len(inplis)<1:
	print namedat	return np.nan
	return list(zip(*[inplis[i:] for i in range(n)]))

In [8]:
#Clean data frame and generate ngrams
def gramsplitter(df,n):
	eng_stop_words = set(stopwords.words("english")+['test','1234','various','addresses'])
	fr_stop_words = set(stopwords.words("french"))
	ger_stop_words = set(stopwords.words("german"))
	stop_wrds = eng_stop_words.union(fr_stop_words).union(ger_stop_words)
	
	def gramclean(x):
		if x.isnull().all():
			return np.nan
		return ' '.join([word for word in x.str.replace('[^\w\s]','').str.split().tolist()[0] if word not in stop_wrds])

	df = df.apply(gramclean,axis=1)
	return df.map(lambda x: x if (isinstance(x,float)) else ngrammer(x.split(" "),n))

In [9]:
def analyze(dat):
	#Get name and address data
	namedat = dat[get_relevcols(dat,'name')]
	addr_dat = dat[get_relevcols(dat,'address')]

	#exploretext(namedat,"name")
	
	#exploretext(addr_dat,"addr")
	
	#Create dataframe
	prop_data = pd.DataFrame({"name_bigrams": gramsplitter(namedat,2),
							  "address_trigrams": gramsplitter(addr_dat,3)})

	return prop_data

In [16]:
def createparams(db_props,propname):
    prop_dat= pd.DataFrame(list(db_props.find()))
    #hot_beds = pd.DataFrame(list(db.hotelbeds_properties.find()))
    colrmv = ["verification","verified","mapped_to","mapped_to_ext_id"]
    #prop_dat.drop(colrmv,axis=1,inplace=True)

    prop_subdf = get_df(prop_dat)
    #print prop_subdf.columns
    prop_subdf = prop_subdf.join(prop_dat[colrmv])
    prop_params = analyze(prop_subdf)
    prop_params['latitude'] = prop_subdf[get_relevcols(prop_subdf,'latitude')]
    prop_params['longitude'] = prop_subdf[get_relevcols(prop_subdf,'longitude')]
    prop_params['city'] = prop_subdf['city']
    prop_params[colrmv] = prop_subdf[colrmv]
    print prop_params.head()
    prop_params.to_csv(propname+"_parameters.scsv",encoding='utf-8',index=False,sep=';')
    pk.dump(prop_params,open(propname+"_params.pk","wb"))

In [1]:
def benchmark(db):
    createparams(db.hotelbeds_properties,"hotelbeds")
    createparams(db.taap_properties,"taap")

In [4]:
client = MongoClient()
db = client['supplier_static_database']

In [5]:
def findidx(StemWords,word):
    A = []
    for key,value in StemWords.items():
        if word in value:
            return key
        else:
            return word

In [6]:
def stemmer(x,stemdic,stopwords):
    tokens = x.replace('[^\w\s]','').split()
    for idx,word in enumerate(tokens):
        if word in stopwords:
            del tokens[idx]
            continue
        tokens[idx] = findidx(stemdic.loc[stemdic.index.str.startswith(word[0])],word)
    return tokens            

In [23]:
eng_stop_words = set(stopwords.words("english")+['test','1234','various','addresses'])
fr_stop_words = set(stopwords.words("french"))
ger_stop_words = set(stopwords.words("german"))
stop_words = eng_stop_words.union(fr_stop_words).union(ger_stop_words)
pk.dump(stop_words,open("stop_words.pk","wb"))

In [32]:
prop_subdf= pd.DataFrame(list(db.hotelbeds_properties.find()))
prop_subdf.columns = map(unicode.lower, prop_subdf.columns)

In [85]:
namedat=prop_subdf[get_relevcols(prop_subdf,'name')].iloc[:,0].str.lower()
print namedat

0                                         inter-burgo ansan
1                                          dogo glory condo
2                                                   w hotel
3                                        onyang grand hotel
4                                   onyang hot spring hotel
5              onyang cheil hotel (ex. onyang palace hotel)
6                  springhill suites paso robles atascadero
7                              best western plus colony inn
8                                                    hesse 
9                                   ibis styles aachen city
10                                pullman aachen quellenhof
11                               mercure aachen europaplatz
12                                    leonardo hotel aachen
13                                    aquis grana cityhotel
14                                              buschhausen
15                                      novotel aachen city
16                        domicil reside

In [103]:
stemingdic = pd.read_csv("stemming_dict.csv",header=None,index_col=False)
stemingdic.index = stemingdic[0]
stemingdic = stemingdic.iloc[:,1]

In [105]:
namedat.apply(stemmer,args=(stemingdic,stop_words))

0                                      [inter-burgo, ansan]
1                                      [dogo, glory, condo]
2                                                [w, hotel]
3                                    [onyang, grand, hotel]
4                              [onyang, hot, spring, hotel]
5         [onyang, cheil, hotel, None, onyang, palace, h...
6            [springhill, suites, paso, robles, atascadero]
7                        [best, western, plus, colony, inn]
8                                                   [hesse]
9                              [ibis, styles, aachen, city]
10                                  [pullman, aachen, None]
11                           [mercure, aachen, europaplatz]
12                                [leonardo, hotel, aachen]
13                                [aquis, grana, cityhotel]
14                                            [buschhausen]
15                                  [novotel, aachen, city]
16                  [domicil, residenz, 

In [1]:
def gramclean(x,stop_words):
		if x.isnull().all():
			return np.nan
		return ' '.join([word for word in x.str.replace('[^\w\s]','').str.split().tolist()[0] if word not in stop_words])